In [1]:
!git clone https://github.com/alneremin/lung-cancer-diagnosis.git

Cloning into 'lung-cancer-diagnosis'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 57 (delta 12), reused 52 (delta 9), pack-reused 0
Unpacking objects: 100% (57/57), done.


In [3]:
!git -C lung-cancer-diagnosis checkout develop

Branch 'develop' set up to track remote branch 'develop' from 'origin'.
Switched to a new branch 'develop'


In [1]:
import sys
import os

sys.path.insert(0, os.path.join("lung-cancer-diagnosis","src"))
!pip install pydicom

In [1]:
!rm -r path_to_download
!rm -r lung-cancer-diagnosis/src/downloads

In [34]:
import sys
sys.path.remove("lung-cancer-diagnosis/src")

In [2]:
import load_data
from importlib import reload
import os
import sys
import TCIA.tciaclientimpl as tciaclientimpl

reload(tciaclientimpl)
classfile = os.path.join(sys.path[0], "VisualizationTools","category.txt")
annotation_path = os.path.join(sys.path[0], "downloads", "Annotation")
path_to_download = os.path.join(sys.path[0], "downloads")
load_data.download_data(classfile, annotation_path, path_to_download)

https://services.cancerimagingarchive.net/services/v3/TCIA/query/getPatient?Collection=Lung-PET-CT-Dx&format=json

Query TCIA - getPatient(Lung-PET-CT-Dx, JSON)
https://services.cancerimagingarchive.net/services/v3/TCIA/query/getPatientStudy?Collection=Lung-PET-CT-Dx&PatientID=Lung_Dx-A0001&format=json

Query TCIA - getPatientStudy(Lung-PET-CT-Dx,Lung_Dx-A0001, JSON)
https://services.cancerimagingarchive.net/services/v3/TCIA/query/getSeries?Collection=Lung-PET-CT-Dx&StudyInstanceUID=1.3.6.1.4.1.14519.5.2.1.6655.2359.165554066086145834377508507990&PatientID=Lung_Dx-A0001&format=json

Query TCIA - getSeries(Lung-PET-CT-Dx, 1.3.6.1.4.1.14519.5.2.1.6655.2359.165554066086145834377508507990, Lung_Dx-A0001, Null, Null, Null, Null, Null, JSON)
https://services.cancerimagingarchive.net/services/v3/TCIA/query/getSOPInstanceUIDs?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.6655.2359.213534032021332276911485641315&format=json
{'SeriesInstanceUID': '1.3.6.1.4.1.14519.5.2.1.6655.2359.213534032021332276

In [ ]:
# code below taken from https://github.com/1297rohit/VGG16-In-Keras

In [ ]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
trdata = ImageDataGenerator()
directory = os.path.join("lung-cancer-diagnosis","src", "downloads", "Lung-PET-CT-Dx")
traindata = trdata.flow_from_directory(directory=directory,target_size=(224,224))
tsdata = ImageDataGenerator()

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=2, activation="softmax"))

In [ ]:
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')

In [ ]:
hist = model.fit_generator(steps_per_epoch=100,generator=traindata, validation_data= testdata, validation_steps=10,epochs=100,callbacks=[checkpoint,early])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history["acc"])
plt.plot(hist.history['val_acc'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [ ]:
from keras.preprocessing import image
img = image.load_img("",target_size=(224,224))
img = np.asarray(img)
plt.imshow(img)
img = np.expand_dims(img, axis=0)
from keras.models import load_model
saved_model = load_model("vgg16_1.h5")
output = saved_model.predict(img)

In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/VGG16.ipynb" /content/lung-cancer-diagnosis/VGG16.ipynb

In [42]:
!git -C lung-cancer-diagnosis add .

In [45]:
!git -C lung-cancer-diagnosis commit -m "changed load_data" -m "added relative path parameter"

[develop 946ad05] changed load_data
 2 files changed, 11 insertions(+), 9 deletions(-)


In [44]:
!git -C lung-cancer-diagnosis config --global user.email "alneremin@ya.ru"
!git -C lung-cancer-diagnosis config --global user.name "alneremin"

In [ ]:
!git -C lung-cancer-diagnosis push -u origin develop

Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 2.23 KiB | 2.23 MiB/s, done.
Total 3 (delta 0), reused 0 (delta 0)
To https://github.com/alneremin/lung-cancer-diagnosis.git
   4e5d900..1b40e96  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.


In [ ]:
!git -C lung-cancer-diagnosis remote rm origin